In [ ]:
#CNN

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Check if GPU is available and set the device accordingly
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define transformations for training and validation data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load your dataset using torchvision's ImageFolder
data_dir = '/kaggle/input/final-data/FinalData'
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split the dataset into training, testing, and validation sets
train_size = 0.7
test_size = 0.15
val_size = 0.15
train_dataset, test_val_dataset = train_test_split(dataset, train_size=train_size, shuffle=True, random_state=42)
test_dataset, val_dataset = train_test_split(test_val_dataset, train_size=test_size/(test_size + val_size), shuffle=True, random_state=42)

# Create separate data loaders for training and validation
train_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Define data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Define the CNN model architecture
class CNN(nn.Module):
    def _init_(self):
        super(CNN, self)._init_()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer
        self.fc2 = nn.Linear(512, 4)  # Output classes: 4

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)
        x = torch.relu(self.fc1(x))
        features = self.dropout(x)  # Apply dropout
        x = self.fc2(features)
        return x, features

# Initialize the CNN model
model_cnn = CNN()

# Define loss function and optimizer
criterion_cnn = nn.CrossEntropyLoss()
weight_decay = 0.001  # L2 regularization (weight decay)
optimizer_cnn = optim.Adam(model_cnn.parameters(), lr=0.001, weight_decay=weight_decay)

# Move the model to GPU if available
model_cnn.to(device)

# Initialize variables to store the highest accuracy and corresponding metrics
highest_accuracy = 0.0
best_epoch = 0
best_confusion_matrix = None
total_correct = 0
total_samples = 0
num_epochs = 100
average_f1_score = 0.0  # Initialize average F1 score
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    model_cnn.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer_cnn.zero_grad()
        outputs, _ = model_cnn(inputs)
        loss = criterion_cnn(outputs, labels)
        loss.backward()
        optimizer_cnn.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Print training loss
    train_losses.append(running_loss / len(train_loader))
    train_accuracy = correct / total
    train_accuracies.append(train_accuracy)
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {running_loss / len(train_loader)}, Train Accuracy: {train_accuracy}")

    # Validation
    model_cnn.eval()
    correct = 0
    total = 0
    val_loss = 0.0
    predictions = []
    ground_truths = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs, _ = model_cnn(inputs)
            loss = criterion_cnn(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            predictions.extend(predicted.cpu().numpy())
            ground_truths.extend(labels.cpu().numpy())

        # Calculate accuracy and F1 score for the current epoch
        accuracy = correct / total
        f1 = f1_score(ground_truths, predictions, average='weighted')
        val_losses.append(val_loss / len(val_loader))
        val_accuracies.append(accuracy)

        # Print F1 score
        print(f"F1 Score: {f1}")

        # Accumulate total correct predictions and total samples
        total_correct += correct
        total_samples += total

        # Update average F1 score
        average_f1_score += f1

        # Check if the current accuracy is higher than the highest accuracy
        if accuracy > highest_accuracy:
            highest_accuracy = accuracy
            best_epoch = epoch
            best_confusion_matrix = confusion_matrix(ground_truths, predictions)
            best_f1_score = f1  # Store the F1 score of the best model

            # Save the best model checkpoint
            best_model_path = 'best_cnn_model.pth'
            torch.save(model_cnn.state_dict(), best_model_path)

# Calculate the final average accuracy
final_avg_accuracy = total_correct / total_samples

# Calculate the average F1 score across all epochs
average_f1_score /= num_epochs

print(f"Training Average Accuracy: {100 * final_avg_accuracy:.2f}%")
print(f"Training Average F1 Score: {average_f1_score:.4f}")

# Print the accuracy and F1 score of the best model
print(f"Best Model Accuracy: {highest_accuracy:.4f}")
print(f"Best Model F1 Score: {best_f1_score:.4f}")

# Plot training and validation loss curves
plt.figure(figsize=(10, 6))
plt.plot(val_losses, label='Validation Loss')
plt.title('Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot training and validation accuracy curves
plt.figure(figsize=(10, 6))
plt.plot(val_accuracies, label='Validation Accuracy')
plt.title('Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Load the best model checkpoint for evaluation
model_cnn = CNN()
model_cnn.load_state_dict(torch.load(best_model_path))
model_cnn.to(device)

# Initialize variables to store true labels and predicted probabilities for each class
all_labels = []
all_probabilities = []

# Evaluate the model on the test set to get true labels and predicted probabilities
model_cnn.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs, _ = model_cnn(inputs)
        probabilities = nn.functional.softmax(outputs, dim=1)
        all_labels.extend(labels.cpu().numpy())
        all_probabilities.extend(probabilities.cpu().numpy())

# Convert to numpy arrays
all_labels = np.array(all_labels)
all_probabilities = np.array(all_probabilities)

# Calculate accuracy, f1-score, and confusion matrix for the test set
predictions = np.argmax(all_probabilities, axis=1)
accuracy = np.mean(predictions == all_labels)
f1 = f1_score(all_labels, predictions, average='weighted')
conf_matrix = confusion_matrix(all_labels, predictions)

# Calculate precision, recall, f1-score, and support for each class
report = classification_report(all_labels, predictions, target_names=["Class 0", "Class 1", "Class 2", "Class 3"])

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F1-Score: {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(report)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

# Extract features from the test set for t-SNE visualization
model_cnn.eval()
all_features = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs, features = model_cnn(inputs)
        all_features.extend(features.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

all_features = np.array(all_features)
all_labels = np.array(all_labels)

# Apply t-SNE to the extracted features
tsne = TSNE(n_components=2, random_state=42)
tsne_features = tsne.fit_transform(all_features)

# Plot t-SNE results
plt.figure(figsize=(10, 8))
sns.scatterplot(x=tsne_features[:, 0], y=tsne_features[:, 1], hue=all_labels, palette='viridis', s=50)
plt.title('t-SNE Visualization of CNN Features')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.legend(title='Classes')
plt.show()

In [ ]:
#RESNET18


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Check if GPU is available and set the device accordingly
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define transformations for training and validation data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load your dataset using torchvision's ImageFolder
data_dir = '/kaggle/input/final-data1/FinalData'
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split the dataset into training, testing, and validation sets
train_size = 0.7
test_size = 0.15
val_size = 0.15
train_dataset, test_val_dataset = train_test_split(dataset, train_size=train_size, shuffle=True, random_state=42)
test_dataset, val_dataset = train_test_split(test_val_dataset, train_size=test_size/(test_size + val_size), shuffle=True, random_state=42)

# Create separate data loaders for training and validation
train_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Define data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Initialize the ResNet model
model_resnet = models.resnet18(pretrained=True)

# Modify the fully connected layer (classifier) for the model
num_ftrs = model_resnet.fc.in_features
dropout_prob = 0.5  # Dropout probability
model_resnet.fc = nn.Sequential(
    nn.Dropout(p=dropout_prob),
    nn.Linear(num_ftrs, 4)  # Output size is 4 for 4 classes
)

# Define loss function and optimizer
criterion_resnet = nn.CrossEntropyLoss()
weight_decay = 0.001  # L2 regularization (weight decay)
optimizer_resnet = optim.Adam(model_resnet.parameters(), lr=0.001, weight_decay=weight_decay)

# Move the model to GPU if available
model_resnet.to(device)

# Initialize variables to store the highest accuracy and corresponding metrics
highest_accuracy = 0.0
best_epoch = 0
best_confusion_matrix = None
total_correct = 0
total_samples = 0
num_epochs = 100
average_f1_score = 0.0  # Initialize average F1 score
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    model_resnet.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer_resnet.zero_grad()
        outputs = model_resnet(inputs)
        loss = criterion_resnet(outputs, labels)
        loss.backward()
        optimizer_resnet.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Print training loss
    train_losses.append(running_loss / len(train_loader))
    train_accuracy = correct / total
    train_accuracies.append(train_accuracy)
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {running_loss / len(train_loader)}, Train Accuracy: {train_accuracy}")

    # Validation
    model_resnet.eval()
    correct = 0
    total = 0
    val_loss = 0.0
    predictions = []
    ground_truths = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model_resnet(inputs)
            loss = criterion_resnet(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            predictions.extend(predicted.cpu().numpy())
            ground_truths.extend(labels.cpu().numpy())

        # Calculate accuracy and F1 score for the current epoch
        accuracy = correct / total
        f1 = f1_score(ground_truths, predictions, average='weighted')
        val_losses.append(val_loss / len(val_loader))
        val_accuracies.append(accuracy)

        # Print F1 score
        print(f"F1 Score: {f1}")

        # Accumulate total correct predictions and total samples
        total_correct += correct
        total_samples += total

        # Update average F1 score
        average_f1_score += f1

        # Check if the current accuracy is higher than the highest accuracy
        if accuracy > highest_accuracy:
            highest_accuracy = accuracy
            best_epoch = epoch
            best_confusion_matrix = confusion_matrix(ground_truths, predictions)
            best_f1_score = f1  # Store the F1 score of the best model

            # Save the best model checkpoint
            best_model_path = 'best_resnet_model.pth'
            torch.save(model_resnet.state_dict(), best_model_path)

# Calculate the final average accuracy
final_avg_accuracy = total_correct / total_samples

# Calculate the average F1 score across all epochs
average_f1_score /= num_epochs

print(f"Training Average Accuracy: {100 * final_avg_accuracy:.2f}%")
print(f"Training Average F1 Score: {average_f1_score:.4f}")

# Print the accuracy and F1 score of the best model
print(f"Best Model Accuracy: {highest_accuracy:.4f}")
print(f"Best Model F1 Score: {best_f1_score:.4f}")

# Plot training and validation loss curves
plt.figure(figsize=(10, 6))
plt.plot(val_losses, label='Validation Loss')
plt.title('Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot training and validation accuracy curves
plt.figure(figsize=(10, 6))
plt.plot(val_accuracies, label='Validation Accuracy')
plt.title('Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Load the best model checkpoint for evaluation
model_resnet = models.resnet18(pretrained=False)
model_resnet.fc = nn.Sequential(
    nn.Dropout(p=dropout_prob),
    nn.Linear(num_ftrs, 4)
)
model_resnet.load_state_dict(torch.load(best_model_path))
model_resnet.to(device)

# Initialize variables to store true labels and predicted probabilities for each class
all_labels = []
all_probabilities = []

# Evaluate the model on the test set to get true labels and predicted probabilities
model_resnet.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_resnet(inputs)
        probabilities = nn.functional.softmax(outputs, dim=1)
        all_labels.extend(labels.cpu().numpy())
        all_probabilities.extend(probabilities.cpu().numpy())

# Convert to numpy arrays
all_labels = np.array(all_labels)
all_probabilities = np.array(all_probabilities)

# Calculate accuracy, f1-score, and confusion matrix for the test set
predictions = np.argmax(all_probabilities, axis=1)
accuracy = np.mean(predictions == all_labels)
f1 = f1_score(all_labels, predictions, average='weighted')
conf_matrix = confusion_matrix(all_labels, predictions)

# Calculate precision, recall, f1-score, and support for each class
report = classification_report(all_labels, predictions, target_names=["Class 0", "Class 1", "Class 2", "Class 3"])

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F1-Score: {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(report)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

# t-SNE plot

# Remove the final fully connected layer for feature extraction
feature_extractor = nn.Sequential(*list(model_resnet.children())[:-1])
feature_extractor.to(device)

# Extract features from the test set
features = []
labels = []

model_resnet.eval()
with torch.no_grad():
    for inputs, label in test_loader:
        inputs = inputs.to(device)
        outputs = feature_extractor(inputs)
        outputs = outputs.view(outputs.size(0), -1)  # Flatten the outputs
        features.append(outputs.cpu().numpy())
        labels.extend(label.numpy())

# Convert features and labels to numpy arrays
features = np.concatenate(features, axis=0)
labels = np.array(labels)

# Apply t-SNE to the extracted features
tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(features)

# Plot the t-SNE results
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=tsne_results[:, 0], y=tsne_results[:, 1],
    hue=labels,
    palette=sns.color_palette("hsv", 4),
    legend="full",
    alpha=0.6
)
plt.title('t-SNE Plot')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.legend(title='Classes')
plt.show()

In [ ]:
#SQUEEZENET

#SQUEEZENET

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import classification_report
from sklearn.manifold import TSNE

# Check if GPU is available and set the device accordingly
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define transformations for training and validation data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load your dataset using torchvision's ImageFolder
data_dir = '/kaggle/input/final-data/FinalData'
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split the dataset into training, testing, and validation sets
train_size = 0.7
test_size = 0.15
val_size = 0.15
train_dataset, test_val_dataset = train_test_split(dataset, train_size=train_size, shuffle=True, random_state=42)
test_dataset, val_dataset = train_test_split(test_val_dataset, train_size=test_size/(test_size + val_size), shuffle=True, random_state=42)

# Create separate data loaders for training and validation
train_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Define data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Initialize the SqueezeNet model
model_squeezenet = models.squeezenet1_0(pretrained=True)

# Add dropout layer
model_squeezenet.classifier[1] = nn.Sequential(
    nn.Dropout(p=0.5),  # Increase dropout rate
    nn.Conv2d(512, 4, kernel_size=(1, 1), stride=(1, 1))  # Modify the classification head
)

# Define loss function and optimizer
criterion_squeezenet = nn.CrossEntropyLoss()
weight_decay = 0.001  # L2 regularization (weight decay)
optimizer_squeezenet = optim.Adam(model_squeezenet.parameters(), lr=0.001, weight_decay=weight_decay)

# Move the model to GPU if available
model_squeezenet.to(device)

# Initialize variables to store the highest accuracy and corresponding metrics
highest_accuracy = 0.0
best_epoch = 0
best_confusion_matrix = None

total_correct = 0
total_samples = 0

num_epochs = 100
average_f1_score = 0.0  # Initialize average F1 score
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    model_squeezenet.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer_squeezenet.zero_grad()
        outputs = model_squeezenet(inputs)
        loss = criterion_squeezenet(outputs, labels)
        loss.backward()
        optimizer_squeezenet.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Print training loss
    train_losses.append(running_loss / len(train_loader))
    train_accuracy = correct / total
    train_accuracies.append(train_accuracy)
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {running_loss / len(train_loader)}, Train Accuracy: {train_accuracy}")

    # Validation
    model_squeezenet.eval()
    correct = 0
    total = 0
    val_loss = 0.0
    predictions = []
    ground_truths = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model_squeezenet(inputs)
            loss = criterion_squeezenet(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            predictions.extend(predicted.cpu().numpy())
            ground_truths.extend(labels.cpu().numpy())

        # Calculate accuracy and F1 score for the current epoch
        accuracy = correct / total
        f1 = f1_score(ground_truths, predictions, average='weighted')
        val_losses.append(val_loss / len(val_loader))
        val_accuracies.append(accuracy)

        # Print F1 score
        print(f"F1 Score: {f1}")

        # Accumulate total correct predictions and total samples
        total_correct += correct
        total_samples += total

        # Update average F1 score
        average_f1_score += f1

        # Check if the current accuracy is higher than the highest accuracy
        if accuracy > highest_accuracy:
            highest_accuracy = accuracy
            best_epoch = epoch
            best_confusion_matrix = confusion_matrix(ground_truths, predictions)
            best_f1_score = f1  # Store the F1 score of the best model

            # Save the best model checkpoint
            best_model_path = 'best_squeezenet_model.pth'
            torch.save(model_squeezenet.state_dict(), best_model_path)

# Calculate the final average accuracy
final_avg_accuracy = total_correct / total_samples

# Calculate the average F1 score across all epochs
average_f1_score /= num_epochs

print(f"Training Average Accuracy: {100 * final_avg_accuracy:.2f}%")
print(f"Training Average F1 Score: {average_f1_score:.4f}")

# Print the accuracy and F1 score of the best model
print(f"Best Model Accuracy: {highest_accuracy:.4f}")
print(f"Best Model F1 Score: {best_f1_score:.4f}")

# Plot training and validation loss curves
plt.figure(figsize=(10, 6))
plt.plot(val_losses, label='Validation Loss')
plt.title('Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot training and validation accuracy curves
plt.figure(figsize=(10, 6))
plt.plot(val_accuracies, label='Validation Accuracy')
plt.title('Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Load the best model checkpoint for evaluation
model_squeezenet = models.squeezenet1_0(pretrained=False)
model_squeezenet.classifier[1] = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Conv2d(512, 4, kernel_size=(1, 1), stride=(1, 1))
)
model_squeezenet.load_state_dict(torch.load(best_model_path))
model_squeezenet.to(device)

# Initialize variables to store true labels and predicted probabilities for each class
all_labels = []
all_probabilities = []
all_features = []

# Evaluate the model on the test set to get true labels and predicted probabilities
model_squeezenet.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_squeezenet(inputs)
        probabilities = nn.functional.softmax(outputs, dim=1)
        features = model_squeezenet.features(inputs).cpu().numpy()
        all_labels.extend(labels.cpu().numpy())
        all_probabilities.extend(probabilities.cpu().numpy())
        all_features.extend(features)

# Convert to numpy arrays
all_labels = np.array(all_labels)
all_probabilities = np.array(all_probabilities)
all_features = np.array(all_features)

from sklearn.decomposition import PCA

# Reshape features to 2D array (samples, features)
all_features_flat = all_features.reshape(all_features.shape[0], -1)

# Apply PCA to reduce dimensionality
pca = PCA(n_components=50)  # Adjust the number of components as needed
all_features_pca = pca.fit_transform(all_features_flat)

# Compute t-SNE for visualization
tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(all_features_pca)

# Plot t-SNE visualization
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=tsne_results[:, 0], y=tsne_results[:, 1],
    hue=all_labels,
    palette=sns.color_palette("hsv", len(np.unique(all_labels))),
    legend="full",
    alpha=0.7
)
plt.title('t-SNE Visualization of Test Data')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.legend(title='Class')
plt.show()

# Calculate accuracy, f1-score, and confusion matrix for the test set
predictions = np.argmax(all_probabilities, axis=1)
accuracy = np.mean(predictions == all_labels)
f1 = f1_score(all_labels, predictions, average='weighted')
conf_matrix = confusion_matrix(all_labels, predictions)

# Calculate precision, recall, f1-score, and support for each class
report = classification_report(all_labels, predictions, target_names=["Class 0", "Class 1", "Class 2", "Class 3"])

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F1-Score: {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(report)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()